In [1]:
#import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from keras import backend
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

Using TensorFlow backend.


In [44]:
n_in = 39
n_time = 20
n_hidden = 128
n_out = 20

epochs = 10
batch_size = 20

In [57]:
log_mini= pd.read_csv("../../data/training_set_preproc/log_mini.csv")
sessions= pd.read_csv("../../data/training_set_preproc/session_log_mini.csv")
songs= log_mini.drop(["session_id",'y1', 'y2'], axis=1)
# songs= log_mini.drop(['y2'], axis=1)


X_train = np.array(songs)
#Y_train = np.array(log_mini['y2'])

X_train_c = []
for i in range(int(len(log_mini)/20)):
    X_train_c.append(X_train[20*i:20*(i+1), :])

X_train = np.array(X_train_c)
#X_test = X_test.reshape(X_test.shape[0], 28, 28)

In [46]:
print(log_mini.columns)

Index(['session_id', 'duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability',
       'dyn_range_mean', 'energy', 'flatness', 'instrumentalness', 'liveness',
       'loudness', 'mechanism', 'organism', 'speechiness', 'tempo',
       'time_signature', 'valence', 'acoustic_vector_0', 'acoustic_vector_1',
       'acoustic_vector_2', 'acoustic_vector_3', 'acoustic_vector_4',
       'acoustic_vector_5', 'acoustic_vector_6', 'acoustic_vector_7', 'key_0',
       'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8',
       'key_9', 'key_10', 'mode_major', 'y1', 'y2'],
      dtype='object')


In [47]:
nx = np.array(songs)
n = nx.shape[0]
f = nx.shape[1]

s = int(n/20)

nx1 = np.reshape(nx,(s,20,f),order='C')


print(nx1.shape)
print(nx1[0,:,0])
print(nx.shape)
print(nx[:20,0])

(10000, 20, 41)
['0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e']
(200000, 41)
['0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_

In [48]:
shuffle_indices = np.random.permutation(np.arange(s))
x_rnn_s = nx1[shuffle_indices,:,:]

print(nx1[0,:,0])
print(x_rnn_s[1,:,0])

['0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e'
 '0_00006f66-33e5-4de7-a324-2d18e439fc1e']
['0_0e506bcb-7a0e-43c7-8444-f242ef7e587e'
 '0_0e506bcb-7a0e-43c7-8444-f242ef7e587e'
 '0_0e506bcb-7a0e-43c7-8444-f242ef7e587e'
 '0_0e506bcb-7a0e-43c7-8444-f242e

In [49]:
nfc = np.array(sessions)

print(nfc.shape)

(10000, 11)


In [50]:
Y_train = np.array(log_mini['y2'])
Y_train_c = []
for i in range(int(len(log_mini)/20)):
    Y_train_c.append(Y_train[20*i:20*(i+1)])
Y_train = np.array(Y_train_c)

In [58]:
print(X_train.shape, Y_train.shape, log_mini.shape)

(10000, 20, 39) (10000, 20) (200000, 42)


In [52]:
def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)

In [53]:
def selective_hinge(y_true, y_pred):
    return K.mean(K.maximum(1. - y_true * y_pred, 0.) * K.pow(y_true,2), axis=-1)

In [54]:
def selective_binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.sign(y_pred)), axis=-1)

In [59]:
from keras.models import load_model

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(n_time, n_in)))
model.add(Dense(n_out, kernel_initializer=weight_variable))
model.add(Activation('linear'))

#
#model = load_model("model.h5", custom_objects={'selective_hinge': selective_hinge})

model.compile(loss='hinge',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              metrics=[selective_binary_accuracy])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               86016     
_________________________________________________________________
dense_2 (Dense)              (None, 20)                2580      
_________________________________________________________________
activation_2 (Activation)    (None, 20)                0         
Total params: 88,596
Trainable params: 88,596
Non-trainable params: 0
_________________________________________________________________


In [60]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

hist = model.fit(X_train, Y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(X_train, Y_train),
                 callbacks=[early_stopping])

Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.9619 - selective_binary_accuracy: 0.2355 - val_loss: 0.9504 - val_selective_binary_accuracy: 0.2391
Epoch 2/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.9503 - selective_binary_accuracy: 0.2399 - val_loss: 0.9502 - val_selective_binary_accuracy: 0.2407
Epoch 3/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.9487 - selective_binary_accuracy: 0.2405 - val_loss: 0.9480 - val_selective_binary_accuracy: 0.2416
Epoch 4/10
10000/10000 [==============================] - 12s 1ms/step - loss: 0.9475 - selective_binary_accuracy: 0.2408 - val_loss: 0.9504 - val_selective_binary_accuracy: 0.2417
Epoch 5/10
10000/10000 [==============================] - 9s 947us/step - loss: 0.9463 - selective_binary_accuracy: 0.2416 - val_loss: 0.9445 - val_selective_binary_accuracy: 0.2418
Epoch 6/10
10000/10000 [====================

In [70]:
keys = hist.history.keys()
print(keys)
list_metrics = []

for key in keys:
    print(key)
#     if key.find('val_') > -1 and key.find('val_loss') == -1:
    if key.find('val_') > -1:
        print(key)
        print(key[4:])
        list_metrics.append(key[4:])
    
print(list_metrics)

dict_keys(['selective_binary_accuracy', 'val_loss', 'loss', 'val_selective_binary_accuracy'])
selective_binary_accuracy
val_loss
val_loss
loss
loss
val_selective_binary_accuracy
val_selective_binary_accuracy
selective_binary_accuracy
['loss', 'selective_binary_accuracy']


In [ ]:
ex = 